In [2]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K

In [3]:
class LeNet:
	@staticmethod
	def build(numChannels, imgRows, imgCols, numClasses,
		activation="relu", weightsPath=None):
		# initialize the model
		model = Sequential()
		inputShape = (imgRows, imgCols, numChannels)

		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (numChannels, imgRows, imgCols)
            
		# define the first set of CONV => ACTIVATION => POOL layers
		model.add(Conv2D(20, 5, padding="same",
			input_shape=inputShape))
		model.add(Activation(activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
		# define the second set of CONV => ACTIVATION => POOL layers
		model.add(Conv2D(50, 5, padding="same"))
		model.add(Activation(activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
		# define the first FC => ACTIVATION layers
		model.add(Flatten())
		model.add(Dense(500))
		model.add(Activation(activation))

		# define the second FC layer
		model.add(Dense(numClasses))

		# lastly, define the soft-max classifier
		model.add(Activation("softmax"))
        
		# if a weights path is supplied (inicating that the model was
		# pre-trained), then load the weights
		if weightsPath is not None:
			model.load_weights(weightsPath)

		# return the constructed network architecture
		return model

In [4]:
# import the necessary packages
#from pyimagesearch.cnn.networks.lenet import LeNet
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import argparse
import cv2

In [5]:
# grab the MNIST dataset (if this is your first time running this
# script, the download may take a minute -- the 55MB MNIST dataset
# will be downloaded)
print("[INFO] downloading MNIST...")
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()

# if we are using "channels first" ordering, then reshape the
# design matrix such that the matrix is:
# num_samples x depth x rows x columns
if K.image_data_format() == "channels_first":
	trainData = trainData.reshape((trainData.shape[0], 1, 28, 28))
	testData = testData.reshape((testData.shape[0], 1, 28, 28))

# otherwise, we are using "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
else:
	trainData = trainData.reshape((trainData.shape[0], 28, 28, 1))
	testData = testData.reshape((testData.shape[0], 28, 28, 1))

# scale data to the range of [0, 1]
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

[INFO] downloading MNIST...
11493376/11490434 [==============================] - 32s 3us/step


In [6]:
# transform the training and testing labels into vectors in the
# range [0, classes] -- this generates a vector for each label,
# where the index of the label is set to `1` and all other entries
# to `0`; in the case of MNIST, there are 10 class labels
trainLabels = np_utils.to_categorical(trainLabels, 10)
testLabels = np_utils.to_categorical(testLabels, 10)

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=0.01)
model = LeNet.build(numChannels=1, imgRows=28, imgCols=28,
	numClasses=10,
	weightsPath= None)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [8]:
# only train and evaluate the model if we *are not* loading a
# pre-existing model
#if args["load_model"] < 0:
print("[INFO] training...")
model.fit(trainData, trainLabels, batch_size=128, epochs=20,
	verbose=1)

# show the accuracy on the testing set
print("[INFO] evaluating...")
(loss, accuracy) = model.evaluate(testData, testLabels,
	batch_size=128, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))

[INFO] training...
Epoch 1/20
60000/60000 [==============================] - 511s 9ms/step - loss: 0.9651 - accuracy: 0.7407
Epoch 2/20
60000/60000 [==============================] - 333s 6ms/step - loss: 0.2550 - accuracy: 0.9245
Epoch 3/20
60000/60000 [==============================] - 167s 3ms/step - loss: 0.1802 - accuracy: 0.9463
Epoch 4/20
60000/60000 [==============================] - 167s 3ms/step - loss: 0.1408 - accuracy: 0.9585
Epoch 5/20
60000/60000 [==============================] - 180s 3ms/step - loss: 0.1164 - accuracy: 0.9663
Epoch 6/20
60000/60000 [==============================] - 167s 3ms/step - loss: 0.1002 - accuracy: 0.9707
Epoch 7/20
60000/60000 [==============================] - 168s 3ms/step - loss: 0.0892 - accuracy: 0.9734
Epoch 8/20
60000/60000 [==============================] - 165s 3ms/step - loss: 0.0802 - accuracy: 0.9757
Epoch 9/20
60000/60000 [==============================] - 166s 3ms/step - loss: 0.0732 - accuracy: 0.9781
Epoch 10/20
60000/60000 [==

In [9]:
print("[INFO] dumping weights to file...")
model.save_weights("F:\\STUDY MATERIAL\\DEEP LEARNING\\lenet-mnist\\lenet-mnist\\MY model\\weights", overwrite=True)

[INFO] dumping weights to file...


In [10]:
# randomly select a few testing digits
for i in np.random.choice(np.arange(0, len(testLabels)), size=(10,)):
	# classify the digit
	probs = model.predict(testData[np.newaxis, i])
	prediction = probs.argmax(axis=1)

	# extract the image from the testData if using "channels_first"
	# ordering
	if K.image_data_format() == "channels_first":
		image = (testData[i][0] * 255).astype("uint8")

	# otherwise we are using "channels_last" ordering
	else:
		image = (testData[i] * 255).astype("uint8")

	# merge the channels into one image
	image = cv2.merge([image] * 3)

	# resize the image from a 28 x 28 image to a 96 x 96 image so we
	# can better see it
	image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)

	# show the image and prediction
	cv2.putText(image, str(prediction[0]), (5, 20),
				cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
	print("[INFO] Predicted: {}, Actual: {}".format(prediction[0],
		np.argmax(testLabels[i])))
	cv2.imshow("Digit", image)
	cv2.waitKey(0)
cv2.destroyAllWindows()

[INFO] Predicted: 2, Actual: 2
[INFO] Predicted: 2, Actual: 2
[INFO] Predicted: 4, Actual: 4
[INFO] Predicted: 4, Actual: 4
[INFO] Predicted: 0, Actual: 0
[INFO] Predicted: 3, Actual: 3
[INFO] Predicted: 7, Actual: 7
[INFO] Predicted: 4, Actual: 4
[INFO] Predicted: 3, Actual: 3
[INFO] Predicted: 5, Actual: 5
